In [19]:
def get_tech(url):
    if not url.startswith('http'):
        url = 'https://' + url
    webpage = WebPage.new_from_url(url)
    wappalyzer = Wappalyzer.latest()
    technologies = []
    for tech in wappalyzer.analyze(webpage):
        technologies.append(tech)
    return technologies

In [20]:
get_tech('portal.airbyte.com')

NameError: name 'webPage' is not defined